In [1]:
import sys
import re
import json
import urllib.request
import pandas as pd
import xml.etree.ElementTree as ET
sys.path.insert(0, '../scripts')
from WikiPage import WikiPage

In [2]:
dog_dat = pd.read_csv('../data/Wikipedia-dog.csv', index_col=0)
fish_dat = pd.read_csv('../data/Wikipedia-fish.csv', index_col=0)
dat = pd.concat([dog_dat, fish_dat])
dat.head()

,website,title,page_type,text_raw
970284,https://en.wikipedia.org/wiki/Category:Dog_sho...,Category:Dog shows and showing,category,{{Cat main|Conformation show|Show dog}}\n{{por...
972913,https://en.wikipedia.org/wiki/Category:Dog_health,Category:Dog health,category,This is a collection of articles about the hea...
970251,https://en.wikipedia.org/wiki/Category:Dog_org...,Category:Dog organizations,category,This is an automatically collected list of art...
729436,https://en.wikipedia.org/wiki/Category:Dog_sports,Category:Dog sports,category,This is an automatically accumulated list of a...
978163,https://en.wikipedia.org/wiki/Category:Dogs_as...,Category:Dogs as pets,category,[[Category:Dogs|Pets]]\n[[Category:Mammals as ...


In [3]:
dog_xmlfile = '../data/Wikipedia-dog.xml'
dog_tree = ET.parse(dog_xmlfile)
dog_root = dog_tree.getroot()

fish_xmlfile = '../data/Wikipedia-fish.xml'
fish_tree = ET.parse(fish_xmlfile)
fish_root = fish_tree.getroot()

In [4]:
wp = WikiPage(dog_root[30])
print(wp.text_parentid)
#dat.loc[wp.text_parentid]

685069728


In [5]:
wiki_id = str(wp.text_parentid)
request = "https://en.wikipedia.org/w/api.php?action=query&prop=info&pageids=__pageids__&inprop=url&format=json"
id_request = request.replace("__pageids__", wiki_id)
s = urllib.request.urlopen(id_request).read()
article_dict = json.loads(s)
#print(str(article_dict['query']['pages'][wiki_id]['fullurl']))

In [6]:
article_dict

{'batchcomplete': '',
 'query': {'pages': {'685069728': {'missing': '', 'pageid': 685069728}}}}

In [7]:
def get_url_from_wiki_id(wiki_id):
    wiki_id = str(wiki_id)
    request = "https://en.wikipedia.org/w/api.php?action=query&prop=info&pageids=__pageids__&inprop=url&format=json"
    id_request = request.replace("__pageids__", wiki_id)
    s = urllib.request.urlopen(id_request).read()
    article_dict = json.loads(s)
    return str(article_dict['query']['pages'][wiki_id]['fullurl'])

In [8]:
print(wp.website)

https://en.wikipedia.org/wiki/PDSA_Certificate_for_Animal_Bravery_or_Devotion


In [9]:
text = wp.text_raw
print(text)

{{Infobox award
| name           = PDSA Certificate for Animal Bravery or Devotion
| current_awards = 
| image          = 
| imagesize      = 150px
| alt            = 
| caption        = The PDSA Certificate for Animal Bravery or Devotion
| description    = Animal gallantry and devotion to duty
| presenter      = [[People's Dispensary for Sick Animals]]
| country        =  United Kingdom
| location       = 
| year           = 2001
| year2          = 
| holder         = 
| website        = 
}}

The '''PDSA Certificate for Animal Bravery or Devotion''' is an animal bravery award that acknowledges the extraordinary bravery and devotion to duty of animals.<ref>{{cite web|title=PDSA Report and Accounts 2009|url=http://www.charity-commission.gov.uk/ScannedAccounts/Ends17/0000208217_ac_20091231_e_c.pdf|publisher=Charity Commission|accessdate=8 April 2011|format=PDF}}</ref>   It was created by the [[People's Dispensary for Sick Animals]] (PDSA) in 2001.<ref name=dottypdsa/> So far all of the r

In [10]:
re.sub('\[\[', '', '[[test]]')

'test]]'

In [11]:
print(re.search('\|.+', 'test|remove bar').group())
print(re.sub('\|.+', '', 'test|remove bar'))

|remove bar
test


In [12]:
def find_wiki_links(text):
    searchstr = '(\[[^\[\]]*\[.*?\][^\]\[]*\])'
    link_urls = []
    link_titles = []
    link_titles_raw = re.findall(searchstr, text)
    for link_title_raw in link_titles_raw:
        link_title = re.sub('\|.+', '', link_title_raw)#remove junk at end
        link_title = re.sub('\[\[', '', link_title)#remove brackets
        link_title = re.sub('\]\]', '', link_title)#remove brackets
        link_titles.append(link_title)
        link_title = re.sub(' ', '_',  link_title)
        link_url = 'https://en.wikipedia.org/wiki/%s' % (link_title)
        link_urls.append(link_url)
    return list(set(zip(link_titles, link_urls)))

In [13]:
print(find_wiki_links(text))

[('therapy dog', 'https://en.wikipedia.org/wiki/therapy_dog'), ('Dickin Medal', 'https://en.wikipedia.org/wiki/Dickin_Medal'), ("People's Dispensary for Sick Animals", "https://en.wikipedia.org/wiki/People's_Dispensary_for_Sick_Animals"), ('Category:Domesticated animals', 'https://en.wikipedia.org/wiki/Category:Domesticated_animals'), ('Yorkshire Terrier', 'https://en.wikipedia.org/wiki/Yorkshire_Terrier'), ('guide dog', 'https://en.wikipedia.org/wiki/guide_dog'), ('Dotty (donkey)', 'https://en.wikipedia.org/wiki/Dotty_(donkey)'), ('Labrador Retriever', 'https://en.wikipedia.org/wiki/Labrador_Retriever'), ('Category:Awards to animals', 'https://en.wikipedia.org/wiki/Category:Awards_to_animals'), ('Pit Bull', 'https://en.wikipedia.org/wiki/Pit_Bull'), ('Category:Dogs', 'https://en.wikipedia.org/wiki/Category:Dogs'), ('World War II', 'https://en.wikipedia.org/wiki/World_War_II'), ('Orlando (dog)', 'https://en.wikipedia.org/wiki/Orlando_(dog)'), ('Piper (dog)', 'https://en.wikipedia.org/w

In [14]:
def query_wiki_api(title):
    base_query = 'https://en.wikipedia.org/w/api.php?action=query&titles= &prop=revisions&rvprop=content&format=json'
    query = re.sub(' ', title, base_query)
    query = re.sub(' ', '%20', query)
    result = urllib.request.urlopen(query).read()
    article_dict = json.loads(result)
    return article_dict

def get_text_from_wiki_api(title):
    article_dict = query_wiki_api(title)
    wiki_id = list(article_dict['query']['pages'].keys())[0]
    return article_dict['query']['pages'][wiki_id]['revisions'][0]['*']

In [15]:
test_title = find_wiki_links(text)[0][0]
print(test_title)
test_text = get_text_from_wiki_api(test_title)
print(test_text)

therapy dog
{{medref|date=December 2015}}
[[Image:Golden Retrievers dark and light.jpg|thumb|right|280px|[[Golden Retriever]]s are often used as therapy dogs due to their calm demeanor, gentle disposition, and friendliness to strangers.]]

A '''therapy dog''' is a [[canine]] that might be trained to provide affection and comfort to people in [[hospital]]s, [[retirement home]]s, [[nursing home]]s, [[school]]s, [[hospice]]s, [[disaster area]]s, and to people with mental disorders, such as [[anxiety disorder]] or [[autism]]

Therapy dogs are usually not [[assistance dog|assistance]] or [[service dog|service]] dogs, but can be one or both with some organizations.<ref>{{cite web|title=Frequently Asked Questions about Service Animals and the ADA|url=https://www.ada.gov/regs2010/service_animal_qa.pdf|website=ADA|accessdate=17 January 2017}}</ref>

In the U.S., therapy dogs are not service animals and are not afforded the same privileges as service animals are.<ref>https://adata.org/publicatio

In [16]:
find_wiki_links(test_text)

[('school', 'https://en.wikipedia.org/wiki/school'),
 ('service dog', 'https://en.wikipedia.org/wiki/service_dog'),
 ('canine', 'https://en.wikipedia.org/wiki/canine'),
 ('assistance dog', 'https://en.wikipedia.org/wiki/assistance_dog'),
 ('nursing home', 'https://en.wikipedia.org/wiki/nursing_home'),
 ('American Kennel Club',
  'https://en.wikipedia.org/wiki/American_Kennel_Club'),
 ('anxiety disorder', 'https://en.wikipedia.org/wiki/anxiety_disorder'),
 ('Category:Therapy', 'https://en.wikipedia.org/wiki/Category:Therapy'),
 ('Service dog', 'https://en.wikipedia.org/wiki/Service_dog'),
 ('disaster area', 'https://en.wikipedia.org/wiki/disaster_area'),
 ('hospice', 'https://en.wikipedia.org/wiki/hospice'),
 ('Registered Nurse', 'https://en.wikipedia.org/wiki/Registered_Nurse'),
 ('St. John Ambulance Canada#Therapy Dog',
  'https://en.wikipedia.org/wiki/St._John_Ambulance_Canada#Therapy_Dog'),
 ('hospital', 'https://en.wikipedia.org/wiki/hospital'),
 ('Pets As Therapy', 'https://en.wik